# Function Pointer and Passing functions as arguments to other functions

* Part I: what and how to use function pointers.
    * Pointers to functions
    * Assigning a function to a function pointer
    * Calling a function using a function pointer
* Part II: how to use function pointers in a nice way.
    * native way
    * use alias way
    * use std::function way
* Part III: more about std::function

In [1]:
#include <iostream>

# Part I: what and how to use function pointers.

a pointer is a variable that holds the address of another variable. Function pointers are similar, except that instead of pointing to variables, they point to functions. Consider the following function:
~~~
int foo()
{
    return 5;
}
~~~
Identifier foo is the function’s name. **But what type is the function?** Functions have their own l-value function type -- in this case, a function type that returns an integer and takes no parameters. Much like variables, functions live at an assigned address in memory.

When a function is called (via the () operator), execution jumps to the address of the function being called:

In [2]:
int foo() // code for foo starts at memory address 0x002717f0
{
    return 5;
};

In [3]:
int main()
{
    foo(); // jump to address 0x002717f0
    
    //what if we’ve unintentionally sent function foo directly to std::cout. What happens in this case?
    std::cout << &foo << '\n'; // we meant to call foo(), but instead we're printing foo itself!
    return 0;
};

## 1.1 Pointers to functions

The syntax for creating a non-const function pointer is one of the ugliest things you will ever see in C++:
~~~
// fcnPtr is a pointer to a function that takes no arguments and returns an integer
int (*fcnPtr)();
~~~
In the above snippet, fcnPtr is a pointer to a function that has no parameters and returns an integer. fcnPtr can point to any function that matches this type.
    
The parentheses around *fcnPtr are necessary for precedence reasons, as int* fcnPtr() would be interpreted as a forward declaration for a function named fcnPtr that takes no parameters and returns a pointer to an integer.

To make a const function pointer, the const goes after the asterisk:
~~~
int (*const fcnPtr)();
~~~
If you put the const before the int, then that would indicate the function being pointed to would return a const int.

## 1.2 Assigning a function to a function pointer

Function pointers can be initialized with a function (and non-const function pointers can be assigned a function). In the above example, we have used foo directly, and it has been converted to a function pointer. Like with pointers to variables, we can also use &foo to get a function pointer to foo.

In [ ]:
int foo()
{
    return 5;
}

int goo()
{
    return 6;
}

int main()
{
    int (*fcnPtr)(){ &foo }; // fcnPtr points to function foo
    fcnPtr = &goo; // fcnPtr now points to function goo

    return 0;
}

This would actually assign the return value from a call to function goo() to fcnPtr, which isn’t what we want. We want fcnPtr to be assigned the address of function goo, not the return value from function goo(). So no parentheses are needed.

Note that the type (parameters and return type) of the function pointer must match the type of the function. Here are some examples of this:

In [ ]:
// function prototypes
int foo();
double goo();
int hoo(int x);

// function pointer assignments
int (*fcnPtr1)(){ &foo }; // okay
int (*fcnPtr2)(){ &goo }; // wrong -- return types don't match!
double (*fcnPtr4)(){ &goo }; // okay
fcnPtr1 = &hoo; // wrong -- fcnPtr1 has no parameters, but hoo() does
int (*fcnPtr3)(int){ &hoo }; // okay

Unlike fundamental types, C++ will implicitly convert a function into a function pointer if needed (so you don’t need to use the address-of operator (&) to get the function’s address). However, it will not implicitly convert function pointers to void pointers, or vice-versa.

Function pointers can also be initialized or assigned the value nullptr:

In [5]:
int (*fcnptr)() { nullptr }; // okay

## 1.3 Calling a function using a function pointer
* explicit dereference
* implicit dereference

The other primary thing you can do with a function pointer is use it to actually call the function. There are two ways to do this. The first is via explicit dereference:

In [ ]:
int foo(int x)
{
    return x;
}

int main()
{
    int (*fcnPtr)(int){ &foo }; // Initialize fcnPtr with function foo
    (*fcnPtr)(5); // call function foo(5) through fcnPtr.

    return 0;
}

The second way is via implicit dereference:

In [ ]:
int foo(int x)
{
    return x;
}

int main()
{
    int (*fcnPtr)(int){ &foo }; // Initialize fcnPtr with function foo
    fcnPtr(5); // call function foo(5) through fcnPtr.

    return 0;
}

As you can see, the implicit dereference method looks just like a normal function call -- which is what you’d expect, since normal function names are pointers to functions anyway! However, some older compilers do not support the implicit dereference method, but all modern compilers should.

One interesting note: **Default parameters won’t work for functions called through function pointers.** Default parameters are resolved at compile-time (that is, if you don’t supply an argument for a defaulted parameter, the compiler substitutes one in for you when the code is compiled). However, function pointers are resolved at run-time. Consequently, default parameters can not be resolved when making a function call with a function pointer. You’ll explicitly have to pass in values for any defaulted parameters in this case.

Also note that because function pointers can be set to nullptr, it’s a good idea to assert or conditionally test whether your function pointer is a null pointer before calling it. Just like with normal pointers, dereferencing a null function pointer lead to undefined behavior.

In [ ]:
int foo(int x)
{
    return x;
}

int main()
{
    int (*fcnPtr)(int){ &foo }; // Initialize fcnPtr with function foo
    if (fcnPtr) // make sure fcnPtr isn't a null pointer
        fcnPtr(5); // otherwise this will lead to undefined behavior

    return 0;
}

## 1.3 Passing functions as arguments to other functions

One of the most useful things to do with function pointers is pass a function as an argument to another function. Functions used as arguments to another function are sometimes called callback functions.

Consider a case where you are writing a function to perform a task (such as sorting an array), but you want the user to be able to define how a particular part of that task will be performed (such as whether the array is sorted in ascending or descending order). Let’s take a closer look at this problem as applied specifically to sorting, as an example that can be generalized to other similar problems.

Many comparison-based sorting algorithms work on a similar concept: the sorting algorithm iterates through a list of numbers, does comparisons on pairs of numbers, and reorders the numbers based on the results of those comparisons. Consequently, by varying the comparison, we can change the way the algorithm sorts without affecting the rest of the sorting code.

Here’s a full example of a selection sort that uses a function pointer parameter to do a user-defined comparison, along with an example of how to call it:

In [ ]:
#include <utility> // for std::swap
#include <iostream>

// Note our user-defined comparison is the third parameter
void selectionSort(int* array, int size, bool (*comparisonFcn)(int, int))
{
    // Step through each element of the array
    for (int startIndex{ 0 }; startIndex < (size - 1); ++startIndex)
    {
        // bestIndex is the index of the smallest/largest element we've encountered so far.
        int bestIndex{ startIndex };

        // Look for smallest/largest element remaining in the array (starting at startIndex+1)
        for (int currentIndex{ startIndex + 1 }; currentIndex < size; ++currentIndex)
        {
            // If the current element is smaller/larger than our previously found smallest
            if (comparisonFcn(array[bestIndex], array[currentIndex])) // COMPARISON DONE HERE
            {
                // This is the new smallest/largest number for this iteration
                bestIndex = currentIndex;
            }
        }

        // Swap our start element with our smallest/largest element
        std::swap(array[startIndex], array[bestIndex]);
    }
}

// Here is a comparison function that sorts in ascending order
// (Note: it's exactly the same as the previous ascending() function)
bool ascending(int x, int y)
{
    return x > y; // swap if the first element is greater than the second
}

// Here is a comparison function that sorts in descending order
bool descending(int x, int y)
{
    return x < y; // swap if the second element is greater than the first
}

// This function prints out the values in the array
void printArray(int* array, int size)
{
    for (int index{ 0 }; index < size; ++index)
    {
        std::cout << array[index] << ' ';
    }

    std::cout << '\n';
}

int main()
{
    int array[9]{ 3, 7, 9, 5, 6, 1, 8, 2, 4 };

    // Sort the array in descending order using the descending() function
    selectionSort(array, 9, descending);
    printArray(array, 9);

    // Sort the array in ascending order using the ascending() function
    selectionSort(array, 9, ascending);
    printArray(array, 9);

    return 0;
}

As you can see, using a function pointer in this context provides a nice way to allow a caller to “hook” their own functionality into something you’ve previously written and tested, which helps facilitate code reuse! Previously, if you wanted to sort one array in descending order and another in ascending order, you’d need multiple versions of the sort routine. Now you can have one version that can sort any way the caller desires!

Note: If a function parameter is of a function type, it will be converted to a pointer to the function type. This means
~~~
void selectionSort(int* array, int size, bool (*comparisonFcn)(int, int))
~~~
can be equivalently written as:
~~~
void selectionSort(int* array, int size, bool comparisonFcn(int, int))
~~~
This only works for function parameters, not stand-alone function pointers, and so is of somewhat limited use.

# Part II: how to use function pointers in a nice way.

### 2.1 Making function pointers prettier with type aliases

Let’s face it -- the syntax for pointers to functions is ugly. However, type aliases can be used to make pointers to functions look more like regular variables:
~~~
using ValidateFunction = bool(*)(int, int);
~~~
This defines a type alias called “ValidateFunction” that is a pointer to a function that takes two ints and returns a bool.

Now instead of doing this:
~~~
bool validate(int x, int y, bool (*fcnPtr)(int, int)); // ugly
~~~
You can do this:
~~~
bool validate(int x, int y, ValidateFunction pfcn) // clean
~~~

## 2.2 using std::function
An alternate method of defining and storing function pointers is to use std::function, which is part of the standard library <functional> header. To define a function pointer using this method, declare a std::function object like so:

In [ ]:
#include <functional>
bool validate(int x, int y, std::function<bool(int, int)> fcn); // std::function method that returns a bool and takes two int parameters

As you see, both the return type and parameters go inside angled brackets, with the parameters inside parentheses. If there are no parameters, the parentheses can be left empty.

Updating our earlier example with std::function:

In [ ]:
#include <functional>
#include <iostream>

int foo()
{
    return 5;
}

int goo()
{
    return 6;
}

int main()
{
    std::function<int()> fcnPtr{ &foo }; // declare function pointer that returns an int and takes no parameters
    fcnPtr = &goo; // fcnPtr now points to function goo
    std::cout << fcnPtr() << '\n'; // call the function just like normal

    return 0;
}

In [ ]:
Type aliasing std::function can be helpful for readability:

In [ ]:
using ValidateFunctionRaw = bool(*)(int, int); // type alias to raw function pointer
using ValidateFunction = std::function<bool(int, int)>; // type alias to std::function

Also note that std::function only allows calling the function via implicit dereference (e.g. fcnPtr()), not explicit dereference (e.g. (*fcnPtr)()).

### Type inference for function pointers

Much like the auto keyword can be used to infer the type of normal variables, the auto keyword can also infer the type of a function pointer.

In [ ]:
#include <iostream>

int foo(int x)
{
	return x;
}

int main()
{
	auto fcnPtr{ &foo };
	std::cout << fcnPtr(5) << '\n';

	return 0;
}

This works exactly like you’d expect, and the syntax is very clean. The downside is, of course, that all of the details about the function’s parameters types and return type are hidden, so it’s easier to make a mistake when making a call with the function, or using its return value.

### Conclusion

Function pointers are useful primarily when you want to store functions in an array (or other structure), or when you need to pass a function to another function. Because the native syntax to declare function pointers is ugly and error prone, we recommend using std::function. In places where a function pointer type is only used once (e.g. a single parameter or return value), std::function can be used directly. In places where a function pointer type is used multiple times, a type alias to a std::function is a better choice (to prevent repeating yourself).

# Part III: more about std::function

Class template std::function is a general-purpose polymorphic function wrapper. Instances of std::function can store, copy, and invoke any Callable target -- functions, lambda expressions, bind expressions, or other function objects, as well as pointers to member functions and pointers to data members.

The stored callable object is called the target of std::function. If a std::function contains no target, it is called empty. Invoking the target of an empty std::function results in std::bad_function_call exception being thrown.

In [10]:
#include <functional>
#include <iostream>

In [12]:
struct Foo {
    Foo(int num) : num_(num) {}
    void print_add(int i) const { std::cout << num_+i << '\n'; }
    int num_;
};

In [13]:
void print_num(int i)
{
    std::cout << i << '\n';
};

In [14]:
struct PrintNum {
    void operator()(int i) const
    {
        std::cout << i << '\n';
    }
};

In [15]:
int main()
{
    // store a free function
    std::function<void(int)> f_display = print_num;
    f_display(-9);
 
    // store a lambda
    std::function<void()> f_display_42 = []() { print_num(42); };
    f_display_42();
 
    // store the result of a call to std::bind
    std::function<void()> f_display_31337 = std::bind(print_num, 31337);
    f_display_31337();
 
    // store a call to a member function
    std::function<void(const Foo&, int)> f_add_display = &Foo::print_add;
    const Foo foo(314159);
    f_add_display(foo, 1);
    f_add_display(314159, 1);
 
    // store a call to a data member accessor
    std::function<int(Foo const&)> f_num = &Foo::num_;
    std::cout << "num_: " << f_num(foo) << '\n';
 
    // store a call to a member function and object
    using std::placeholders::_1;
    std::function<void(int)> f_add_display2 = std::bind( &Foo::print_add, foo, _1 );
    f_add_display2(2);
 
    // store a call to a member function and object ptr
    std::function<void(int)> f_add_display3 = std::bind( &Foo::print_add, &foo, _1 );
    f_add_display3(3);
 
    // store a call to a function object(functor)
    std::function<void(int)> f_display_obj = PrintNum();
    f_display_obj(18);
};

In file included from input_line_1:1:
In file included from /home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/new:40:
In file included from /home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/exception:144:
In file included from /home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/bits/nested_exception.h:40:
In file included from /home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/bits/move.h:55:
/home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/type_traits:142:41: error: no member named 'value' in 'std::is_same<void (*)(int), std::function<void
      (int)> >'
    : public __bool_constant<!bool(_Pp::value)>
                                   ~~~~~^
/home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/bits/std_function.h:385:39: note: in instantiat

In file included from /home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/new:40:
In file included from /home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/exception:144:
In file included from /home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/bits/nested_exception.h:40:
In file included from /home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/bits/move.h:55:
/home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/type_traits:1985:36: error: no member named 'value' in 'std::is_array<void (int)>'
           bool _IsArray = is_array<_Up>::value,
                           ~~~~~~~~~~~~~~~^
/home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/type_traits:2009:24: note: in instantiation of default argument for '__decay_selector<void (int)>' required
      her

      std::function<int (const __cling_N511::Foo &)> >'
    : public __bool_constant<!bool(_Pp::value)>
                                   ~~~~~^
/home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/bits/std_function.h:385:39: note: in instantiation of template class 'std::__not_<std::is_same<int
      __cling_N511::Foo::*, std::function<int (const __cling_N511::Foo &)> > >'
      requested here
        using _Requires = typename enable_if<_Cond::value, _Tp>::type;
                                             ^
/home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/bits/std_function.h:445:20: note: in instantiation of template type alias '_Requires' requested here
               typename = _Requires<__not_<is_same<_Functor, function>>, void>,
                          ^
/home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/bits/std_function.h:447:2: note: in instantiation o

/home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/functional:808:5: note: candidate template ignored: substitution failure [with _Func = void
      (__cling_N511::Foo::*)(int) const, _BoundArgs = <const __cling_N511::Foo
      &, const std::_Placeholder<1> &>]
    bind(_Func&& __f, _BoundArgs&&... __args)
    ^
input_line_22:31:47: error: no matching function for call to 'bind'
    std::function<void(int)> f_add_display3 = std::bind( &Foo::print_add...
                                              ^~~~~~~~~
/home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/functional:832:5: note: candidate template ignored: couldn't inferIOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
Note

## the benefit of general-purpose polymorphic function wrapper

In [16]:
using function_type = std::function<void(int)>;
// here we will store all binded functions 
std::vector<function_type> bindings;

// var #1 - you can use simple function
function_type var1 = print_num;
bindings.push_back(var1);
  
// var #2 - you can use lambda function 
function_type var2 = []() { print_num(42); };
bindings.push_back(var2);

for (auto f : bindings)
      f(3);
}

# Reference
* https://www.learncpp.com/cpp-tutorial/function-pointers/;